### Link : https://blog.csdn.net/luoyexuge/article/details/78209670

### Train

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# import data

money = np.array([[109],[82],[99], [72], [87], [78], [86], [84], [94], [57]]).astype(np.float32)
click = np.array([[11], [8], [8], [6],[ 7], [7], [7], [8], [9], [5]]).astype(np.float32)

In [3]:
# split data

x_test = money[0:5].reshape(-1,1)
y_test = click[0:5]
x_train = money[5:].reshape(-1,1)
y_train = click[5:]

In [4]:
# definitions

x = tf.placeholder(tf.float32,[None,1], name='x')
w = tf.Variable(tf.zeros([1, 1]))
b = tf.Variable(tf.zeros([1]))
y = tf.matmul(x, w) + b
tf.add_to_collection('pred_network', y)
y_ = tf.placeholder(tf.float32, [None,1])
cost = tf.reduce_sum(tf.pow((y - y_), 2))
train_step = tf.train.GradientDescentOptimizer(0.000001).minimize(cost)

In [5]:
# training

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
cost_history = []
saver = tf.train.Saver()

for i in range(100):
    feed = {x:x_train, y_:y_train}
    sess.run(train_step, feed_dict = feed)
    cost_history.append(sess.run(cost, feed_dict = feed))

In [6]:
# prediction

print("109的预测值是:", sess.run(y, feed_dict={x: [[109]]}))
print("W_Value: %f" % sess.run(w), "b_Value: %f" % sess.run(b), "cost_Value: %f" % sess.run(cost, feed_dict=feed))
saver_path = saver.save(sess, "./model.ckpt", global_step=100)
print("model saved in file: ", saver_path)

109的预测值是: [[9.848552]]
W_Value: 0.090344 b_Value: 0.001086 cost_Value: 1.043675
model saved in file:  ./model.ckpt-100


一、 saver.restore()时填的文件名，因为在saver.save的时候，每个checkpoint会保存三个文件，如 model.ckpt-100.meta, model.ckpt-100.index, model.ckpt-100.data-00000-of-00001 

在import_meta_graph时填的就是meta文件名，我们知道权值都保存在model.ckpt-100.data-00000-of-00001这个文件中，但是如果在restore方法中填这个文件名，就会报错，应该填的是前缀，这个前缀可以使用tf.train.latest_checkpoint(checkpoint_dir)这个方法获取。

二、模型的y中有用到placeholder，在sess.run()的时候肯定要feed对应的数据，因此还要根据具体placeholder的名字，从graph中使用get_operation_by_name方法获取。

### Test

In [12]:
import tensorflow as tf

sess = tf.Session()
new_saver = tf.train.import_meta_graph('./model.ckpt-100.meta')
new_saver.restore(sess, "./model.ckpt-100")
graph = tf.get_default_graph()
x = graph.get_operation_by_name('x').outputs[0]
y = tf.get_collection("pred_network")[0]
    
print(x, y)

INFO:tensorflow:Restoring parameters from ./model.ckpt-100
Tensor("x:0", shape=(?, 1), dtype=float32) Tensor("add:0", shape=(?, 1), dtype=float32)


In [13]:
print("109的预测值是:", sess.run(y, feed_dict={x: [[109]]}))

109的预测值是: [[9.848552]]


### Reference

https://blog.csdn.net/thriving_fcl/article/details/71423039

https://blog.csdn.net/ying86615791/article/details/72731372

https://blog.csdn.net/huachao1001/article/details/78501928